# 0 - Aprendizagem Supervisionada: Modelo de Classificação

Dados obtidos no site Kaggle:https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling

RowNumber: Identificador único para cada registro, não irá contribuir para a análise.

CustomerId: identificador único para cada cliente.

Surname: Sobrenome de cada cliente.

CreditScore: crédito de um cliente com base em seu histórico de crédito e comportamento financeiro.

Geography: Pais onde cada cliente mora.

Gender: Gênero do cliente(Feminino ou Masculino)

Age: Idade do clientes

Tenure: Quantidade de tempo que o cliente está com o banco.

Balance: O saldo co cliente, ou seja a quantidade de dinheiro na conta bancária .

NumOfProducts: Pode incluir várias ofertas, como contas de poupança, empréstimos, cartões de crédito, entre outros.

HasCrCard: Usada para identificar se o cliente tem ou não cartão de crédito com (1) para caso ele possua, e (0) caso contrario.

IsActiveMember: Indica se o cliente é membro ativo (1) ou não (0) dentro do banco.

EstimatedSalary: Fornece uma aproximação do nível de renda do cliente.

Exited: Indica se um cliente cancelou (1) ou não (0) do banco.

# 1 - Pré-processamento das variaveis categoricas

Temos como o objetivo desse projeto criar um algoritmo de aprendizado de maquina, para prever se um clientes de determinado banco irá irá fazer Churn da empresa, ou seja, cancelar sua conta.

In [1]:
# Instalação do Algoritmo
!pip install lightgbm
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

In [3]:
df = pd.read_csv('Churn_Modelling_Tratado.csv', sep=';', encoding='utf-8')

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
df.shape

(10000, 14)

Removendo colunas que não serão necessárias para o modelo de predição

In [7]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Tenure', 'EstimatedSalary'], axis = 1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CreditScore     10000 non-null  int64  
 1   Geography       10000 non-null  object 
 2   Gender          10000 non-null  object 
 3   Age             10000 non-null  int64  
 4   Balance         10000 non-null  float64
 5   NumOfProducts   10000 non-null  int64  
 6   HasCrCard       10000 non-null  int64  
 7   IsActiveMember  10000 non-null  int64  
 8   Exited          10000 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 703.2+ KB


## 1.1 - LabelEncoder: transformação de variáveis categóricas em numéricas

O LabelEncoder é um método de pré-processamento que pode converter variáveis categóricas em números. A razão pela qual muitos algoritmos de aprendizado de máquina não podem lidar diretamente com variáveis categóricas é que isso é crucial. O LabelEncoder transforma cada categoria individual em uma variável categórica com um número inteiro. No entanto, você deve ter em mente que o LabelEncoder não deve ser usado em variáveis categóricas ordinais. Isso pode levar o modelo a entender a ordem dos valores de forma errada.

Definindo as variáveis previsoras

In [9]:
previsores = df.iloc[:, 0:8].values

In [10]:
previsores

array([[619, 'France', 'Female', ..., 1, 1, 1],
       [608, 'Spain', 'Female', ..., 1, 0, 1],
       [502, 'France', 'Female', ..., 3, 1, 0],
       ...,
       [709, 'France', 'Female', ..., 1, 0, 1],
       [772, 'Germany', 'Male', ..., 2, 1, 0],
       [792, 'France', 'Female', ..., 1, 1, 0]], dtype=object)

Definindo a variável alvo

In [11]:
alvo = df.iloc[:, 8].values

In [12]:
alvo

array([1, 0, 1, ..., 1, 1, 0])

In [13]:
previsores.shape, alvo.shape

((10000, 8), (10000,))

Transformando as variáveis categóricas em númericas

In [14]:
previsores[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])
previsores[:, 2] = LabelEncoder().fit_transform(previsores[:, 2])

In [15]:
previsores

array([[619, 0, 0, ..., 1, 1, 1],
       [608, 2, 0, ..., 1, 0, 1],
       [502, 0, 0, ..., 3, 1, 0],
       ...,
       [709, 0, 0, ..., 1, 0, 1],
       [772, 1, 1, ..., 2, 1, 0],
       [792, 0, 0, ..., 1, 1, 0]], dtype=object)

In [16]:
previsores.shape

(10000, 8)

## 1.2 - OneHotEncoder: Criação de váriaveis Dummy

As variáveis que não podem ser medidas numericamente, como a cor dos olhos ou o tipo de carro, são chamadas de variáveis categóricas. Os algoritmos de machine learning exigem que todas as entradas sejam numéricas, então são criadas variáveis dummy para mostrar as variáveis categóricas como números.

In [17]:
previsores2 = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 1, 2, 3, 4, 5, 6, 7])], remainder='passthrough').fit_transform(previsores)

In [18]:
previsores2

<10000x6925 sparse matrix of type '<class 'numpy.float64'>'
	with 80000 stored elements in Compressed Sparse Row format>

In [19]:
previsores2.shape

(10000, 6925)

## 1.3 - Escalonamento

Agora será necessário fazer o escalonamento, devido ao fato de que muitos algoritmos de machine learning usam distâncias entre pontos de dados para determinar a similaridade entre eles, a normalização é crucial. As características podem ter um peso maior do que outras se suas escalas não forem iguais. Isso pode resultar em resultados errôneos ou imprecisos.

In [20]:
previsores2_esc = StandardScaler(with_mean=False).fit_transform(previsores2)

In [21]:
previsores2_esc.shape

(10000, 6925)

## 1.4 - Separando em Treino e Teste

A divisão do conjunto de dados em treino e teste é fundamental para avaliar o desempenho do modelo. O modelo é ajustado aos dados por meio do conjunto de treinamento, enquanto o desempenho do modelo em dados não vistos é avaliado pelo conjunto de teste. O overfitting, que ocorre quando o modelo se ajusta muito bem aos dados de treinamento mas não generaliza bem para novos dados, pode ser evitado separando os dados de teste e de treinamento. A separação dos dados em treinamento e teste também ajuda na avaliação de vários modelos e na seleção do modelo mais adequado para os dados.

In [28]:
X_treino, X_teste, y_treino, y_teste = train_test_split(previsores2, alvo, test_size=0.2, random_state = 1, stratify = alvo)

In [29]:
X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape

((8000, 6925), (2000, 6925), (8000,), (2000,))

## 1.5 - Dataframes

Previsores: Transformada as variáveis categóricas em númericas, aplicado apenas o Label Encoder.

Previsores2: aplicada o OneHotEncoder, para criar Dummies das variáveis.

previsores2_esc: São as previsores2, porém estão escalonadas(Normalizadas).

# 2 - Modelos Machine Learning

## 2.1 - Naive Bayes: Modelo Não consegue se adaptar ao problema

O algoritmo de classificação Naive Bayes, que se baseia no Teorema de Bayes, é um classificador probabilístico. Ele é usado para organizar itens em diferentes categorias com base em suas características. O algoritmo assume que todas as características contribuem igualmente para a probabilidade de uma categoria específica e que as características são independentes umas das outras. Por exemplo, o Naive Bayes pode ser usado para classificar e-mails como spam ou não spam usando as palavras-chave presentes no e-mail.

Treino

In [ ]:
#Criando o modelo
naive = GaussianNB()
naive.fit(X_treino, y_treino)

GaussianNB()

Avaliação

In [ ]:
#Fazendo as previsões e salvando na variável "previsoes_naive"
previsao_naive = naive.predict(X_teste)
previsao_naive

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
#Verificando a acurácia do modelo
accuracy_score(y_teste, previsao_naive)

0.79

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste, previsao_naive)

array([[1547,   46],
       [ 374,   33]])

In [ ]:
#Relatório de classificação.
print(classification_report(y_teste, previsao_naive))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88      1593
           1       0.42      0.08      0.14       407

    accuracy                           0.79      2000
   macro avg       0.61      0.53      0.51      2000
weighted avg       0.73      0.79      0.73      2000



análise dados de treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino"
previsao_treino = naive.predict(X_treino)
previsao_treino

array([1, 0, 1, ..., 0, 0, 0])

In [ ]:
accuracy_score(y_treino, previsao_treino)

0.782125

In [ ]:
confusion_matrix(y_treino, previsao_treino)

array([[6150,  220],
       [1523,  107]])

### 2.1.1 - Validação Cruzada

In [ ]:
#Separando os dados em Folds
kfold = KFold(n_splits = 10, shuffle=True, random_state = 1)

In [ ]:
# Criando o modelo e salvando os resultados na variável "resultado_naive"
modelo_naive = GaussianNB()
resultado_naive = cross_val_score(modelo_naive, previsores, alvo, cv = kfold)
resultado_naive

array([0.78 , 0.784, 0.785, 0.769, 0.819, 0.785, 0.763, 0.792, 0.771,
       0.777])

In [ ]:
print('Acurácia Média: {:.2f}%'.format(resultado_naive.mean() * 100))

Acurácia Média: 78.25%


Naive Bayes = Modelo Não se adapta ao problema

## 2.2 - Máquinas de vetor de suporte(SVM): 85,4% (treino e teste) e 85,45% (validação cruzada)

Uma coleção de técnicas de aprendizado supervisionado conhecidas como máquinas de vetor de suporte (SVMs) são usadas para classificação e análise de regressão, após a análise de dados e a identificação de padrões. As SVMs podem ser usadas para tarefas de regressão e classificação. Elas são uma classe de aprendizagem supervisionado que foi bem estudada. As SVMs são usadas para uma variedade de tarefas, incluindo a recuperação de informações e a classificação de imagens e texto.

In [ ]:
# Definir os parâmetros que você desejamos testar
param_svm = {'C': [0.1, 1, 1.2], 'kernel': ['linear', 'rbf'], 'gamma': [0.1, 1, 10], 'degree': [2, 3, 4]}

In [ ]:
#Criando um modelo de SVM para o GridSearch
svc = SVC()

In [ ]:
# Criando um objeto GridSearchCV
grid_svc = GridSearchCV(svc, param_svm)

In [ ]:
# Treinando o modelo com o conjunto de treinamento
grid_svc.fit(X_treino, y_treino)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 1.2], 'degree': [2, 3, 4],
                         'gamma': [0.1, 1, 10], 'kernel': ['linear', 'rbf']})

In [ ]:
# melhores parâmetros encontrados pelo Grid Search
print(grid_svc.best_params_)

{'C': 1.2, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}


In [48]:
#Criando e Treinando o modelo
svm = SVC(kernel='rbf', random_state=1, C = 1.2, gamma = 0.1, degree = 2)
svm.fit(X_treino, y_treino)

SVC(C=1.2, degree=2, gamma=0.1, random_state=1)

In [46]:
#Fazendo as previsões e salvando na variável "previsoes_svm"
previsoes_svm = svm.predict(X_teste)
previsoes_svm

array([0, 0, 0, ..., 0, 0, 0])

In [32]:
y_teste

array([0, 0, 0, ..., 1, 0, 0])

In [33]:
#Verificando a acurácia do modelo
accuracy_score(y_teste, previsoes_svm)

0.854

In [34]:
#Matriz de confusão
confusion_matrix(y_teste, previsoes_svm)

array([[1558,   35],
       [ 257,  150]])

In [35]:
#Relatório de classificação
print(classification_report(y_teste, previsoes_svm))

              precision    recall  f1-score   support

           0       0.86      0.98      0.91      1593
           1       0.81      0.37      0.51       407

    accuracy                           0.85      2000
   macro avg       0.83      0.67      0.71      2000
weighted avg       0.85      0.85      0.83      2000



análise dados de treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_svm"
previsao_treino_svm = svm.predict(X_treino)
previsao_treino_svm

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_treino

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
accuracy_score(y_treino, previsao_treino_svm)

0.8925

### 2.2.3 - Validação Cruzada

In [ ]:
#Separando os dados em Folds
kfold = KFold(n_splits=10, shuffle = True, random_state = 1)

In [ ]:
# Criando o modelo e salvando os resultados na variável "resultado_svm"
modelo_svm = SVC(kernel='rbf', random_state=1, C = 1.2, gamma = 0.1, degree = 2)
resultado_svm = cross_val_score(modelo_svm, previsores2, alvo, cv = kfold)
resultado_svm

array([0.849, 0.86 , 0.857, 0.861, 0.862, 0.847, 0.84 , 0.854, 0.873,
       0.842])

In [ ]:
print('Acurácia Média: {:.2f}%'.format(resultado_svm.mean() * 100))

Acurácia Média: 85.45%


SVM = 85,4% (treino e teste) e 85,45% (validação cruzada) sem escalonar dados previsores2, com o escalonamento o modelo overfita.

## 2.3 - Regressão Logistica: 85,1% (treino e teste) e 85,12% (validação cruzada)

O modelo estatístico conhecido como regressão logística é usado para prever variáveis binárias, ou seja, variáveis que podem assumir apenas dois valores, como zero ou um, sim ou não, verdadeiro ou falso, etc. A regressão logística, por exemplo, pode ser usada para prever se um consumidor vai comprar um produto, se um e-mail é spam ou não, se um paciente está doente ou não, etc.

In [ ]:
#Criando um modelo de regressão logística para o GridSearch
logistica = LogisticRegression()

In [ ]:
#Definindo os hiperparametros
param_logistica = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'max_iter': [100, 150, 200]}

In [ ]:
#Criando o GridSearch
grid_logistica = GridSearchCV(logistica, param_logistica)

In [ ]:
#Treinamento com o GridSearch
grid_logistica.fit(X_treino, y_treino)

In [ ]:
#Imprimindo os melhores hiperparametros para o modelo
print(grid_logistica.best_params_)

{'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


In [ ]:
print(grid_logistica.best_score_)

0.8493749999999999


In [ ]:
#Criando um modelo de regressão logística
reg_log = LogisticRegression(random_state=1, max_iter=100, penalty='l1', solver = 'liblinear', C = 1)

In [ ]:
#Treinando o modelo
reg_log.fit(X_treino, y_treino)

LogisticRegression(C=1, penalty='l1', random_state=1, solver='liblinear')

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_log"
previsao_log = reg_log.predict(X_teste)
previsao_log

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
#Verificando a acurácia do modelo
accuracy_score(y_teste, previsao_log)

0.851

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste, previsao_log)

array([[1524,   69],
       [ 229,  178]])

In [ ]:
#Relatório de classificação
print(classification_report(y_teste, previsao_log))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91      1593
           1       0.72      0.44      0.54       407

    accuracy                           0.85      2000
   macro avg       0.80      0.70      0.73      2000
weighted avg       0.84      0.85      0.84      2000



Análise com dados de Treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_log"
previsao_treino_log = reg_log.predict(X_treino)

In [ ]:
accuracy_score(y_treino, previsao_treino_log)

0.861875

In [ ]:
confusion_matrix(y_treino, previsao_treino_log)

array([[6107,  263],
       [ 842,  788]])

### 2.3.1 - Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits=10, shuffle = True, random_state = 1)

In [ ]:
# Criando o modelo
modelo_log = LogisticRegression(random_state = 5, max_iter=100, penalty='l1', solver = 'liblinear', C = 1)

In [ ]:
# Salvando os resultados na variável "resultado_log"
resultado_log = cross_val_score(modelo_log, previsores2, alvo, cv = kfold)

In [ ]:
print('Acurácia Média: {:.2f}%'.format(resultado_log.mean() * 100.0))

Acurácia Média: 85.10%


Regressão Logistica = 85,1% (treino e teste) e 85,12% (validação cruzada), com o Dataframe previsores2.

## 2.4 - KNN: 82,85% (treino e teste) e 82,85% (validação cruzada)

O KNN é um tipo de algoritmo de aprendizado supervisionado que é usado para resolver problemas de classificação ou regressão. É conhecido como K-nearest neighbors, ou vizinhos mais próximos. Ele se baseia no conceito de que pontos semelhantes tendem an estar próximos uns dos outros no espaço de atributos.

O mecanismo de funcionamento do KNN é o seguinte: quando queremos classificar ou prever um novo ponto no conjunto de treinamento, ele usa uma medida de distância, como a distância euclidiana, para encontrar os K pontos mais próximos desse ponto. Em seguida, dependendo do tipo de problema, ele usa esses pontos para fazer uma média ou uma votação.

In [ ]:
#Criando um modelo KNN para o GridSearch
knn = KNeighborsClassifier()

In [ ]:
#Definindo os hiperparametros
param_knn = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
             'metric': ['euclidean', 'manhattan','minkowski']}

In [ ]:
#Criando o GridSearch
grid_knn = GridSearchCV(knn, param_knn)

In [ ]:
#Treinamento com o GridSearch
grid_knn.fit(X_treino, y_treino)

In [ ]:
#Imprimindo os melhores hiperparametros para o modelo
print(grid_knn.best_params_)

{'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


In [ ]:
#Criando um modelo KNN
knn1 = KNeighborsClassifier(n_neighbors=7, metric = 'manhattan')

In [ ]:
#Treinando o modelo
knn1.fit(X_treino, y_treino)

KNeighborsClassifier(metric='manhattan', n_neighbors=7)

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_knn"
previsao_knn = knn1.predict(X_teste)
previsao_knn

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
#Verificando a acurácia do modelo
accuracy_score(y_teste, previsao_knn)

0.8285

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste, previsao_knn)

array([[1516,   77],
       [ 266,  141]])

In [ ]:
##Relatório de classificação.
print(classification_report(y_teste, previsao_knn))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      1593
           1       0.65      0.35      0.45       407

    accuracy                           0.83      2000
   macro avg       0.75      0.65      0.67      2000
weighted avg       0.81      0.83      0.81      2000



Análise dados Treino

In [ ]:
modelo_knn = KNeighborsClassifier(n_neighbors=11, metric = 'euclidean')

In [ ]:
modelo_knn.fit(X_treino, y_treino)

KNeighborsClassifier(metric='euclidean', n_neighbors=11)

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_knn"
previsao_treino_knn = modelo_knn.predict(X_treino)

In [ ]:
accuracy_score(y_treino, previsao_treino_knn)

0.8485

In [ ]:
confusion_matrix(y_treino, previsao_treino_knn)

array([[6178,  192],
       [1020,  610]])

### 2.4.1 - Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 30, shuffle=True, random_state = 1)

In [ ]:
# Criando o modelo
modelo_knn = KNeighborsClassifier(n_neighbors=7, metric = 'manhattan')

In [ ]:
# Salvando os resultados na variável "resultado_knn"
resultado_knn = cross_val_score(modelo_knn, previsores2, alvo, cv = kfold)
resultado_knn

array([0.81137725, 0.82335329, 0.80538922, 0.81437126, 0.84730539,
       0.83532934, 0.85329341, 0.80838323, 0.82335329, 0.83532934,
       0.84084084, 0.82882883, 0.85285285, 0.86786787, 0.82882883,
       0.82882883, 0.81981982, 0.83183183, 0.82282282, 0.7957958 ,
       0.83483483, 0.83783784, 0.82882883, 0.81681682, 0.86186186,
       0.82282282, 0.85885886, 0.81081081, 0.8048048 , 0.8018018 ])

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado_knn.mean() * 100.0))

Acurácia Média: 82.85%


Knn = 82,85% (treino e teste) e 82,85% (validação cruzada), com o Dataframe previsores2.

## 2.5 - Árvores de Decisão: 86,53% (treino e teste) e 85,7% (validação cruzada)

Uma árvore de decisão é um tipo de algoritmo de aprendizado supervisionado que é usado para resolver problemas de classificação ou regressão. Ele pode prever categorias discretas (por exemplo, sim ou não) ou valores numéricos (por exemplo, o preço de uma casa).

A árvore de decisão funciona da seguinte maneira: um dado novo que queremos classificar ou prever começa pelo nó raiz e segue o ramo correspondente à resposta da pergunta ou condição do nó. Em seguida, ele repete o procedimento no nó subsequente até chegar a um nó folha que contém o valor previsto ou a classe.


In [ ]:
#Definindo os hiperparametros
param_arvore = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
                'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'sqrt', 'log2']}

In [ ]:
#Criando o modelo de arvore de decisão para o GridSearch
arvored = DecisionTreeClassifier()

In [ ]:
#Criando o GridSearch
grid_arvore = GridSearchCV(arvored, param_grid=param_arvore)

In [ ]:
#Treinamento com o GridSearch
grid_arvore.fit(X_treino, y_treino)

In [ ]:
#Imprimindo os melhores hiperparametros para o modelo
print(grid_arvore.best_params_)

{'criterion': 'gini', 'max_depth': 7, 'max_features': 'log2', 'min_samples_split': 4}


In [ ]:
#Criando o modelo
arvore = DecisionTreeClassifier(criterion='entropy',max_features = 'log2', max_depth = 8, min_samples_split = 7, random_state = 1)

In [ ]:
#Treinando o modelo de arvore de decisão
arvore.fit(X_treino, y_treino)

DecisionTreeClassifier(criterion='entropy', max_depth=8, max_features='log2',
                       min_samples_split=7, random_state=1)

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_arvore"
previsao_arvore = arvore.predict(X_teste)
previsao_arvore

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
#Verificando a acurácia do modelo
accuracy_score(y_teste, previsao_arvore)

0.858

In [ ]:
#Matris de confusão
confusion_matrix(y_teste, previsao_arvore)

array([[1533,   60],
       [ 224,  183]])

Análise dados de Treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_arvore"
previsao_treino_arvore = arvore.predict(X_treino)
previsao_treino_arvore

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
accuracy_score(y_treino, previsao_treino_arvore)

0.861875

In [ ]:
confusion_matrix(y_treino, previsao_treino_arvore)

array([[6179,  191],
       [ 914,  716]])

### 2.5.1 - Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 10, shuffle=True, random_state = 1)

In [ ]:
#Criando o modelo
modelo_arvore = DecisionTreeClassifier(criterion='entropy',max_features = 'log2', max_depth = 8, min_samples_split = 7, random_state = 1)

In [ ]:
# Salvando os resultados na variável "resultado_arvore"
resultado_arvore = cross_val_score(modelo_arvore, previsores, alvo, cv = kfold)
resultado_arvore

array([0.848, 0.858, 0.85 , 0.848, 0.853, 0.857, 0.839, 0.845, 0.863,
       0.84 ])

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado_arvore.mean() * 100.0))

Acurácia Média: 85.01%


Arvores de Decisão = 86,53% (treino e teste) e 85,7% (validação cruzada), com o conjunto de dados previsores.

## 2.6 - Random Forest: 86,15% (treino e teste) e 85,74% (validação cruzada)

Floresta aleatória, também conhecida como "floresta aleatória", é um tipo de algoritmo de aprendizado supervisionado usado para resolver problemas de regressão ou classificação, além de servir como uma árvore de decisão. Por outro lado, ele cria uma "floresta" com um conjunto de árvores de decisão separadas que foram criadas aleatoriamente.

Para fazer uma previsão com um bosque aleatório, cada árvore de decisão faz sua própria previsão. No caso de regressão ou classificação, os resultados são determinados pela média (no caso de regressão) ou pela votação majoritária (no caso de classificação). Assim, o bosque aleatório pode registrar a variabilidade dos dados e evitar o problema de overfitting (ou seja, o modelo se adaptar demais aos dados de treino e perder a capacidade de generalizar para dados novos).

In [ ]:
#criando o modelo
forest = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state = 1, max_depth=8)

In [ ]:
#Treinando o modelo
forest.fit(X_treino, y_treino)

RandomForestClassifier(criterion='entropy', max_depth=8, random_state=1)

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_forest"
previsao_forest = forest.predict(X_teste)
previsao_forest

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
#VErificando a acurácia do modelo
accuracy_score(y_teste, previsao_forest)

0.8615

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste, previsao_forest)

array([[1563,   30],
       [ 247,  160]])

In [ ]:
#Relatório de classificação.
print(classification_report(y_teste, previsao_forest))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      1593
           1       0.84      0.39      0.54       407

    accuracy                           0.86      2000
   macro avg       0.85      0.69      0.73      2000
weighted avg       0.86      0.86      0.84      2000



Análise dados Treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_forest"
previsao_treino_forest = forest.predict(X_treino)

In [ ]:
accuracy_score(y_treino, previsao_treino_forest)

0.871

In [ ]:
confusion_matrix(y_treino, previsao_treino_forest)

array([[6276,   94],
       [ 938,  692]])

### 2.6.1 - Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits=10, shuffle = True, random_state = 1)

In [ ]:
#Criando o modelo
modelo_forest = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state = 1, max_depth=8)

In [ ]:
# Salvando os resultados na variável "resultado_forest"
resultado_forest = cross_val_score(modelo_forest, previsores, alvo, cv = kfold)
resultado_forest

array([0.855, 0.865, 0.857, 0.856, 0.867, 0.865, 0.843, 0.851, 0.875,
       0.846])

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado_forest.mean() * 100.0))

Acurácia Média: 85.80%


Random Forest = 86,15% (treino e teste) e 85,74% (validação cruzada), com o conjunto de dados previsores.

## 2.7 - XGBOOST: 86,5% (treino e teste) e 86,41% (validação cruzada)

XGBoost (também conhecido como eXtreme Gradient Boosting) é um algoritmo de aprendizado supervisionado que usa gradiente boosting para regressão e classificação.

O incremento gradativo é uma abordagem que cria um modelo "forte" e sólido por meio da combinação de vários modelos "fracos", normalmente árvores de decisão. O aumento do gradiente usa o algoritmo de descida do gradiente para reduzir a função de perda (perda) enquanto novos modelos são adicionados à coleção.


In [ ]:
#Criando o modelo para ser usado no GridSearch
xgbg = XGBClassifier()

In [ ]:
#Definindo os hiperparametros
param_xgb = {'max_depth': [3, 4, 5], 'learning_rate': [0.1, 0.01], 'n_estimators': [100, 200], 'min_child_weight': [1, 5],
             'gamma': [0.5, 1], 'subsample': [0.6, 0.8], 'colsample_bytree': [0.6, 0.8], 'objective': ['binary:logistic']}

In [ ]:
#Criando o GridSearch
grid_xgb = GridSearchCV(xgbg, param_xgb)

In [ ]:
#Treinamento com o GridSearch
grid_xgb.fit(X_treino, y_treino)

In [ ]:
#Imprimindo os melhores hiperparametros
print(grid_xgb.best_params_)

In [ ]:
#Criando o modelo
xgb = XGBClassifier(n_estimators = 200, colsample_bytree = 0.6, gamma = 1, learning_rate = 0.1, max_depth = 4,
                    min_child_weight = 5, objective ='binary:logistic', subsample = 0.8, random_state = 1)

In [ ]:
#Treinando o modelo XGBOOST
xgb.fit(X_treino, y_treino)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_xgb"
previsao_xgb = xgb.predict(X_teste)
previsao_xgb

In [ ]:
y_teste

In [ ]:
#Verificando a acurácia
accuracy_score(y_teste, previsao_xgb)

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste, previsao_xgb)

In [ ]:
#Relatório de classificação.
print(classification_report(y_teste, previsao_xgb))

Análise dados de Treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_xgb"
previsao_treino_xgb = xgb.predict(X_treino)

In [ ]:
accuracy_score(y_treino, previsao_treino_xgb)

In [ ]:
confusion_matrix(y_treino, previsao_treino_xgb)

### 2.7.1 - Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 10, shuffle=True, random_state = 5)

In [ ]:
#Criando o modelo
modelo_xgb = XGBClassifier(n_estimators = 200, colsample_bytree = 0.6, gamma = 1, learning_rate = 0.1, max_depth = 4,
                    min_child_weight = 5, objective ='binary:logistic', subsample = 0.8, random_state = 5)

In [ ]:
# Salvando os resultados na variável "resultado_xgb"
resultado_xgb = cross_val_score(modelo_xgb, previsores, alvo, cv = kfold)

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado_xgb.mean() * 100.0))

XGBOOST = 86,5% (treino e teste) e 86,41% (validação cruzada), com o conjunto de dados previsores.

## 2.8 - LIGHTGBM: 86,45% (treino e teste) e 86,24% (validação cruzada)

LightGBM, originalmente desenvolvida pela Microsoft, é uma estrutura de aumento de gradiente distribuída gratuita e de código aberto para aprendizado de máquina. Ele é usado para tarefas de aprendizado de máquina como classificação e classificação, baseado em algoritmos de árvore de decisão.

In [ ]:
#Definindo os hiperparametros
param_lgb = {'learning_rate': [0.01, 0.1], 'n_estimators': [20, 40], 'max_depth': [3, 5],
             'num_leaves': [7, 15], 'min_child_samples': [10, 20], 'subsample': [0.5, 1],
             'colsample_bytree': [0.5, 1], 'objective': ['binary', 'multiclass']}

In [ ]:
#Criando o modelo para ser usado no GridSearch
lgb_model = lgb.LGBMClassifier()

In [ ]:
#Criando o GridSearch
grid_lgb = GridSearchCV(lgb_model, param_lgb)

In [ ]:
#Treinamento com o GridSearch
grid_lgb.fit(X_treino, y_treino)

In [ ]:
#Imprimindo os melhores hiperparametros
print(grid_lgb.best_params_)

In [ ]:
#Criando o modelo
lgbm = lgb.LGBMClassifier(colsample_bytree = 1, learning_rate = 0.1, max_depth = 5, min_child_samples = 20,
                          n_estimators = 40, num_leaves = 15, objective = 'binary', subsample = 0.5, random_state = 1)

In [ ]:
#Treinamento o modelo LIGHTGBM
lgbm.fit(X_treino, y_treino)

LGBMClassifier(colsample_bytree=1, max_depth=5, n_estimators=40, num_leaves=15,
               objective='binary', random_state=1, subsample=0.5)

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_lgbm"
previsao_lgbm = lgbm.predict(X_teste)
previsao_lgbm

In [ ]:
#Verificando a acurácia
accuracy_score(y_teste, previsao_lgbm)

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste, previsao_lgbm)

In [ ]:
#Relatório de classificação.
print(classification_report(y_teste, previsao_lgbm))

Análise dados de Treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_lgbm"
previsao_treino_lgbm = lgbm.predict(X_treino)
previsao_treino_lgbm

In [ ]:
accuracy_score(y_treino, previsao_treino_lgbm)

In [ ]:
confusion_matrix(y_treino, previsao_treino_lgbm)

### 2.8.1 - Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 10, shuffle=True, random_state = 5)

In [ ]:
#Criando o modelo
modelo_lgbm = lgb.LGBMClassifier(colsample_bytree = 1, learning_rate = 0.1, max_depth = 5, min_child_samples = 20,
                          n_estimators = 40, num_leaves = 15, objective = 'binary', subsample = 0.5, random_state = 5)

In [ ]:
# Salvando os resultados na variável "resultado_lgbm"
resultado_lgbm = cross_val_score(modelo_lgbm, previsores, alvo, cv = kfold)
resultado_lgbm

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado_lgbm.mean() * 100.0))

XGBOOST = 86,45% (treino e teste) e 86,24% (validação cruzada), com o conjunto de dados previsores.

## 2.9 - CATBOOST: 86,5% (treino e teste) e 86,50% (validação cruzada)

CatBoost é um algoritmo de aumento de gradiente baseado em árvores que treina redes neurais profundas. Ajustando os pesos de acordo com a distribuição dos dados e incorporando conhecimento prévio do conjunto de dados, ele aumenta a precisão. Isso pode melhorar o desempenho da generalização e reduzir o overfitting.

In [ ]:
#Análisando informações do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CreditScore     10000 non-null  int64  
 1   Geography       10000 non-null  object 
 2   Gender          10000 non-null  object 
 3   Age             10000 non-null  int64  
 4   Balance         10000 non-null  float64
 5   NumOfProducts   10000 non-null  int64  
 6   HasCrCard       10000 non-null  int64  
 7   IsActiveMember  10000 non-null  int64  
 8   Exited          10000 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 703.2+ KB


In [ ]:
#Criando o Dataframe com as variaveis previsores, nomeando como previsores3
previsores3 = df.iloc[:, 0:8]

In [ ]:
#Visualizando o Dataframe
previsores3.head()

,CreditScore,Geography,Gender,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember
0,619,France,Female,42,0.00,1,1,1
1,608,Spain,Female,41,83807.86,1,0,1
2,502,France,Female,42,159660.80,3,1,0
3,699,France,Female,39,0.00,2,0,0
4,850,Spain,Female,43,125510.82,1,1,1


In [ ]:
#Criando um array com a variavel que desejo prever, nomeando como alvo3
alvo3 = df.iloc[:, 8]

In [ ]:
alvo3

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [ ]:
#Fazendo a divisão em treino e teste
X_treino3, X_teste3, y_treino3, y_teste3 = train_test_split(previsores3, alvo3, test_size=0.2, random_state=1, stratify=alvo3)

In [ ]:
#Selecionando as variaveis categoricas
categoricas = ['Geography','Gender']

In [ ]:
#Criando o modelo
catboost = CatBoostClassifier(task_type='CPU', iterations=100, learning_rate=0.1, depth = 8, random_state = 1,
                              eval_metric="Accuracy")

In [ ]:
#Treinando o modelo
catboost.fit( X_treino3, y_treino3, cat_features= categoricas, plot=True, eval_set=(X_teste3, y_teste3))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8501250	test: 0.8560000	best: 0.8560000 (0)	total: 70.8ms	remaining: 7.01s
1:	learn: 0.8526250	test: 0.8565000	best: 0.8565000 (1)	total: 89.8ms	remaining: 4.4s
2:	learn: 0.8540000	test: 0.8600000	best: 0.8600000 (2)	total: 113ms	remaining: 3.67s
3:	learn: 0.8553750	test: 0.8610000	best: 0.8610000 (3)	total: 129ms	remaining: 3.1s
4:	learn: 0.8543750	test: 0.8610000	best: 0.8610000 (3)	total: 143ms	remaining: 2.71s
5:	learn: 0.8538750	test: 0.8585000	best: 0.8610000 (3)	total: 152ms	remaining: 2.37s
6:	learn: 0.8537500	test: 0.8610000	best: 0.8610000 (3)	total: 180ms	remaining: 2.39s
7:	learn: 0.8525000	test: 0.8590000	best: 0.8610000 (3)	total: 215ms	remaining: 2.48s
8:	learn: 0.8538750	test: 0.8595000	best: 0.8610000 (3)	total: 235ms	remaining: 2.37s
9:	learn: 0.8547500	test: 0.8625000	best: 0.8625000 (9)	total: 249ms	remaining: 2.24s
10:	learn: 0.8561250	test: 0.8625000	best: 0.8625000 (9)	total: 257ms	remaining: 2.08s
11:	learn: 0.8550000	test: 0.8630000	best: 0.8630000 

In [ ]:
#Fazendo as previsões e salvando na variável "previsao_cat"
previsao_cat = catboost.predict(X_teste3)
previsao_cat

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste3

4466    0
3754    0
9601    0
9758    0
8577    1
       ..
7788    0
1650    0
1276    1
1043    0
8350    0
Name: Exited, Length: 2000, dtype: int64

In [ ]:
#VErificando a acurácia do modelo
accuracy_score(y_teste3, previsao_cat)

0.8655

In [ ]:
#Matriz de confusão
confusion_matrix(y_teste3, previsao_cat)

array([[1548,   45],
       [ 224,  183]])

In [ ]:
#Relatório de classificação.
print(classification_report(y_teste3, previsao_cat))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1593
           1       0.80      0.45      0.58       407

    accuracy                           0.87      2000
   macro avg       0.84      0.71      0.75      2000
weighted avg       0.86      0.87      0.85      2000



Análise dados de Treino

In [ ]:
#Fazendo as previsões para os dataset de treino e salvando na variável "previsao_treino_cat"
previsao_treino_cat = catboost.predict(X_treino3)
previsao_treino_cat

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
accuracy_score(y_treino3, previsao_treino_cat)

0.8685

In [ ]:
confusion_matrix(y_treino3, previsao_treino_cat)

array([[6180,  190],
       [ 862,  768]])

### 2.9.1 - Validação Cruzada

In [50]:
# Separando os dados em folds
kfold = KFold(n_splits = 10, shuffle=True, random_state = 1)

In [ ]:
#Criando o modelo
modelo_cat = CatBoostClassifier(task_type='CPU', iterations=100, learning_rate=0.1, depth = 8, random_state = 5,
                              eval_metric="Accuracy")

In [ ]:
# Salvando os resultados na variável "resultado_cat"
resultado_cat = cross_val_score(modelo_cat, previsores, alvo, cv = kfold)

In [ ]:
resultado_cat

array([0.868, 0.87 , 0.861, 0.869, 0.871, 0.862, 0.845, 0.861, 0.883,
       0.856])

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado_cat.mean() * 100))

Acurácia Média: 86.46%


XGBOOST = 86,55% (treino e teste) e 86,46% (validação cruzada), com o conjunto de dados previsores.

# Comparando os Modelos

In [ ]:
#Selecionando os modelos que melhor se ajustaram ao dataframe previsores
modelos_previsores = [naive, arvore, forest, xgb, lgbm]

In [ ]:
#Selecionando os modelos que melhor se ajustaram ao dataframe previsores2
modelos_previsores2 = [svm, reg_log, knn1]

In [ ]:
print('Modelos que se adequaram melhor ao Dataframe Previsores, onde aplicada o OneHotEncoder, para criar Dummies das variáveis.\n')

for modelo in modelos_previsores2:
    previsao2 = modelo.predict(X_teste)
    accuracy = accuracy_score(y_teste, previsao2)
    print('Acuracia do modelo {}: {:.2f}%'.format(modelo.__class__.__name__, accuracy*100))
    print('Matriz de Confusão: ''\n')
    print(confusion_matrix(y_teste, previsao2),'\n')

Modelos que se adequaram melhor ao Dataframe Previsores, onde foram transformadas as variáveis categóricas em númericas, aplicado apenas o Label Encoder.
Acuracia do modelo SVC: 85.40%
Matriz de Confusão: 

[[1558   35]
 [ 257  150]] 

Acuracia do modelo LogisticRegression: 85.10%
Matriz de Confusão: 

[[1524   69]
 [ 229  178]] 

Acuracia do modelo KNeighborsClassifier: 82.85%
Matriz de Confusão: 

[[1516   77]
 [ 266  141]] 



In [ ]:
print('Modelos que se adequaram melhor ao Dataframe Previsores, onde foram transformadas as variáveis categóricas em númericas, aplicado apenas o Label Encoder.\n')

for modelo in modelos_previsores:
    previsao1 = modelo.predict(X_teste)
    accuracy = accuracy_score(y_teste, previsao1)
    print('Acuracia do modelo {}: {:.2f}%'.format(modelo.__class__.__name__, accuracy*100))
    print('Matriz de Confusão:''\n')
    print(confusion_matrix(y_teste, previsao1),'\n')

Modelos que se adequaram melhor ao Dataframe Previsores, onde foram transformadas as variáveis categóricas em númericas, aplicado apenas o Label Encoder.

Acuracia do modelo GaussianNB: 79.00%
Matriz de Confusão:

[[1547   46]
 [ 374   33]] 

Acuracia do modelo DecisionTreeClassifier: 85.80%
Matriz de Confusão:

[[1533   60]
 [ 224  183]] 

Acuracia do modelo RandomForestClassifier: 86.15%
Matriz de Confusão:

[[1563   30]
 [ 247  160]] 

Acuracia do modelo XGBClassifier: 86.25%
Matriz de Confusão:

[[1530   63]
 [ 212  195]] 

Acuracia do modelo LGBMClassifier: 86.70%
Matriz de Confusão:

[[1552   41]
 [ 225  182]] 



In [ ]:
catboost.fit( X_treino3, y_treino3, cat_features= categoricas, plot=True, eval_set=(X_teste3, y_teste3))
previsaocat = catboost.predict(X_teste3)
acuraciacat = accuracy_score(y_teste3, previsaocat)

In [ ]:
print('O modelo CATBOOST se adequa melhor ao Dataframe Previsores3, onde não se aplica nada, irá usar as variáveis categóricas')
print('Acuracia do modelo CATBOOST: {:.2f}%'.format(acuraciacat*100))
print('Matriz de Confusão:''\n')
print(confusion_matrix(y_teste, previsaocat),'\n')

O modelo CATBOOST se adequa melhor ao Dataframe Previsores3, onde não se aplica nada, irá usar as variáveis categóricas
Acuracia do modelo CATBOOST: 86.55%
Matriz de Confusão:

[[1548   45]
 [ 224  183]] 



O modelo XGBClassifier é o melhor modelo dentre todos, mesmo que fique com uma acurácia um pouco menor que os modelos LGBMClassifier e DecisionTreeClassifier, ele apresenta uma vantagem que é uma maior quantidade de acertos para os clientes que irão dar Churn, como isso é o que estamos procurando no modelo, então o XGBClassifier é o que melhor se adapta ao nosso problema.

